In [1]:
import xspec as x
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from astropy.io import fits



In [2]:

#Load relxill and test it

# --- STEP 1: DEFINE PATHS ---
# Path to the folder where you compiled relxill (contains lmodel.dat)
relxill_dir = "/home/kyle/software/relxill"

# Path to the folder containing the heavy FITS tables (often the same dir)
relxill_tables = "/home/kyle/software/relxill"

# --- STEP 2: SET ENVIRONMENT VARIABLE ---
# This tells relxill where to look for the tables so you don't have to copy them
os.environ["RELXILL_TABLE_PATH"] = relxill_tables

# --- STEP 3: LOAD THE MODEL ---
# This mimics the "lmod relxill /path" command
print("Loading Relxill...")
x.AllModels.lmod("relxill", relxill_dir)

# --- STEP 4: USE IT ---
# Now you can use 'relxill', 'relxillCp', 'relxilllp', etc.
# Example: simple relativistic reflection
m = x.Model("tbabs * relxillcp")

# Verify it loaded
print(m.componentNames)

Loading Relxill...
Model package relxill successfully loaded.
['TBabs', 'relxillCp']
tbvabs Version 2.3
Cosmic absorption with grains and H2, modified from
Wilms, Allen, & McCray, 2000, ApJ 542, 914-924
Questions: Joern Wilms
joern.wilms@sternwarte.uni-erlangen.de
joern.wilms@fau.de

http://pulsar.sternwarte.uni-erlangen.de/wilms/research/tbabs/

PLEASE NOTICE:
To get the model described by the above paper
you will also have to set the abundances:
   abund wilm

Note that this routine ignores the current cross section setting
as it always HAS to use the Verner cross sections as a baseline.
 *** loading RELXILL model (version 2.5) *** 

Model TBabs<1>*relxillCp<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   relxillCp  Incl       deg      30.0000      frozen
   3    2   relxillCp  a                   0.998000     +/-  0.0          
   4    2   relxillCp  Rin  

In [3]:
# --- 1. SETUP & DATA LOADING ---
# 1. Clear any existing data/models
x.AllData.clear()
x.AllModels.clear()
# 2. Define your file paths for clarity
rgs1_spec = "../products/0865600201/rgs/P0865600201R1S004SRSPEC1001.FIT"
rgs2_spec = "../products/0865600201/rgs/P0865600201R1S004SRSPEC2001.FIT"

# for spec_file in [rgs1_spec, rgs2_spec]:
#     with fits.open(spec_file, mode='update') as hdul:
#         hdul[1].header['POISSERR'] = True
#     print(f"Updated {spec_file}")
#
# for spec_file in [rgs1_spec, rgs2_spec]:
#     with fits.open(spec_file) as hdul:
#         poisserr = hdul[1].header.get('POISSERR', 'NOT FOUND')
#         print(f"{spec_file}: POISSERR = {poisserr}")

rgs1_resp= "../products/0865600201/rgs/P0865600201R1S004RSPMAT1001.FIT"
rgs2_resp = "../products/0865600201/rgs/P0865600201R1S004RSPMAT2001.FIT"

In [4]:
# 3. Load data into separate Data Groups (1:1 and 2:2)
# Using the f-string here mimics "data 1:1 file1 2:2 file2"
x.AllData(f"1:1 {rgs1_spec} 2:2 {rgs2_spec}")


# 4. Assign Response Matrices manually
s1 = x.AllData(1) # Handle for RGS1
s2 = x.AllData(2) # Handle for RGS2

s1.response = rgs1_resp
s2.response = rgs2_resp

# Set Global XSPEC Settings
x.Plot.device = "/null"
x.Plot.xAxis = "keV" # Or "angstrom" for RGS
x.Xset.xsect = "vern"
x.Xset.abund = "wilm"

# --- 2. MODEL DEFINITION ---
# Defining the model with 2 Data Groups creates 2 sets of parameters.
# Set 2 (for RGS2) is automatically linked to Set 1.
m1 = x.Model("constant*tbfeo*(thcomp * bbodyrad + diskbb)")
m2 = x.AllModels(2)


2 spectra  in use
 
Spectral Data File: ../products/0865600201/rgs/P0865600201R1S004SRSPEC1001.FIT  Spectrum 1
Net count rate (cts/s) for Spectrum:1  1.422e-01 +/- 1.293e-03
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-2775
  Telescope: XMM Instrument: RGS1  Channel Type: PI
  Exposure Time: 8.507e+04 sec
 Using fit statistic: chi
 No response loaded.

Spectral Data File: ../products/0865600201/rgs/P0865600201R1S004SRSPEC2001.FIT  Spectrum 2
Net count rate (cts/s) for Spectrum:2  6.300e-02 +/- 8.606e-04
 Assigned to Data Group 2 and Plot Group 2
  Noticed Channels:  1-2775
  Telescope: XMM Instrument: RGS1  Channel Type: PI
  Exposure Time: 8.507e+04 sec
 Using fit statistic: chi
 No response loaded.

***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
Response successfully loaded.
***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
Response successfully loaded.
 Cros

In [5]:


# Grab the first spectrum (RGS1)
s1 = x.AllData(1)

# 1. Check Exposure Time
exposure = s1.exposure
print(f"Exposure Time: {exposure} seconds")

# 2. Check the Raw Data Values (XSPEC stores these as Rates)
# Let's look at the first 5 non-zero bins to see what they look like
rates = [val for val in s1.values if val > 0]
#print(f"Sample Rates (counts/sec): {rates}")

# 3. Calculate Actual Counts
# Counts = Rate * Exposure
counts = [r * exposure for r in rates]
print(f"Sample calculated Counts: {counts[:5]}")
print(f"Total Counts: {sum(counts)}")

# 4. The "Poisson Test"
# Are these calculated counts integers (e.g., 1.0, 2.0) or messy floats (1.023, 5.44)?
is_integer = all(abs(c - round(c)) < 1e-5 for c in counts)

if is_integer:
    print("\n--- DIAGNOSIS ---")
    print("GOOD: Your data corresponds to integer counts.")
    print("The error 'Source file is not Poisson' is likely a HEADER issue.")
    print("FIX: You need to change the 'POISSERR' keyword in your FITS file to True.")
else:
    print("\n--- DIAGNOSIS ---")
    print("BAD: Your data are NOT integers even after multiplying by exposure.")
    print("This means your spectrum has been background-subtracted or flux-calibrated.")
    print("FIX: You MUST use 'chi2' (Chi-Squared) statistic. You cannot use cstat.")


Exposure Time: 85065.015625 seconds
Sample calculated Counts: [1.0, 2.0, 1.0, 2.0, 2.0]
Total Counts: 12093.0

--- DIAGNOSIS ---
GOOD: Your data corresponds to integer counts.
The error 'Source file is not Poisson' is likely a HEADER issue.
FIX: You need to change the 'POISSERR' keyword in your FITS file to True.


In [6]:

m1(1).values = 1.0 # constant
m1(1).frozen = True
m1(2).values = 0.22
m1(2).frozen = True

m1(3).frozen = False
m1(4).frozen = False


m1(6).values = 1.5
m1(6).frozen = False

m1(7).values = 30

m1(8).values = 1
m1(8).frozen = True

m2(1).link = ""
m2(1).frozen = False
m2(1).values = 1.0 # Start at 1.0


Fit statistic  : Chi-Squared                10235.22     using 2775 bins, spectrum 1, group 1.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number: 1
                 Chi-Squared                 5201.51     using 2775 bins, spectrum 2, group 2.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number: 2
Total fit statistic                         15436.72     with 5541 d.o.f.

Test statistic : Chi-Squared                15436.72     using 5550 bins.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number(s): 1 2 

 Null hypothesis probability of 0.00e+00 with 5541 degrees of freedom
 Current data and model not fit yet.

Fit statistic  : Chi-Squared                10235.22     using 2775 bins, spectrum 1, group 1.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number: 1
                 Chi-Squar

In [7]:
# --- 4. FITTING ---
x.Fit.statMethod = "cstat"
x.Fit.query = "yes"
x.Fit.perform()

Default fit statistic is set to: C-Statistic
   This will apply to all current and newly loaded spectra.

Fit statistic  : C-Statistic                20779.46     using 2775 bins, spectrum 1, group 1.
                 C-Statistic                 8141.16     using 2775 bins, spectrum 2, group 2.
Total fit statistic                         28920.62     with 5541 d.o.f.

Test statistic : Chi-Squared                48252.35     using 5550 bins.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number(s): 1 2 

 Null hypothesis probability of 0.00e+00 with 5541 degrees of freedom
 Current data and model not fit yet.
                                   Parameters
C-Statistic  |beta|/N    Lvl           3:O          4:Fe   6:Gamma_tau        7:kT_e         10:kT       11:norm        12:Tin       13:norm     14:factor
19334        511.671       0      0.920932      0.793348       1.54516       34.2167       2.93968      0.509735       1.00627      0.

In [13]:



x.Plot("data")
# Spectrum 1
energy_x   = x.Plot.x(1)      # X-axis values
energy_err = x.Plot.xErr(1)   # X-axis error bars (bin widths)
data_y     = x.Plot.y(1)      # Data counts/rate
data_err   = x.Plot.yErr(1)   # Data errors
model_y    = x.Plot.model(1)  # The folded model values

# Spectrum 2
energy_x_2   = x.Plot.x(2)
energy_err_2 = x.Plot.xErr(2)
data_y_2     = x.Plot.y(2)
data_err_2   = x.Plot.yErr(2)
model_y_2    = x.Plot.model(2)

x.Plot("delchi")
residuals = x.Plot.y(1)
residuals_2 = x.Plot.y(2)

In [14]:
# python
%matplotlib notebook
plt.figure(figsize=(10, 6), dpi=100)
plt.subplot(2, 1, 1)
plt.gca().set_axisbelow(False)  # ensure artists can be above the grid

# Spectrum 1 with larger points
plt.errorbar(
    energy_x, data_y,
    yerr=data_err, xerr=energy_err,
    fmt='o', ms=1,
    elinewidth=0.8, capsize=2,
    ls='-', lw=1, label='Data 1'
)


# Spectrum 2 with smaller points
plt.errorbar(
    energy_x_2, data_y_2,
    yerr=data_err_2, xerr=energy_err_2,
    fmt='o', ms=1,
    elinewidth=0.6, capsize=1,
    color='green', alpha=0.6, ls='', label='Data 2'
)

# Draw models last with higher zorder so they sit above the markers
plt.step(energy_x, model_y, where='mid', color='C1', lw=0.5, zorder=20, label='Model 1')
plt.step(energy_x_2, model_y_2, where='mid', color='darkgreen', lw=0.5, zorder=20, label='Model 2')

plt.yscale("log")
plt.ylabel(r"Counts s$^{-1}$ keV$^{-1}$")
plt.title("Spectrum and Model")
plt.legend()

plt.subplot(2, 1, 2)
plt.step(energy_x, residuals, where='mid', label='Group 1')
plt.step(energy_x_2, residuals_2, where='mid', label='Group 2', color='green')
plt.axhline(0, color='red', linestyle='--')
plt.legend()

plt.xlabel("Energy (keV)")
plt.ylabel(r"Residuals ($\chi$)")
plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

In [15]:
print(data_y_2[6])

0.0


In [16]:


# convert XSPEC plot outputs to numpy arrays
e1   = np.asarray(energy_x)
ee1  = np.asarray(energy_err)
dy1  = np.asarray(data_y)
de1  = np.asarray(data_err)
my1  = np.asarray(model_y)

e2   = np.asarray(energy_x_2)
ee2  = np.asarray(energy_err_2)
dy2  = np.asarray(data_y_2)
de2  = np.asarray(data_err_2)
my2  = np.asarray(model_y_2)

res1 = np.asarray(residuals)
res2 = np.asarray(residuals_2)

# masks: True where data is non-zero (use > 0 or != 0 as desired)
mask1 = dy1 != 0
mask2 = dy2 != 0

# filtered arrays (zeros removed)
e1_f, ee1_f, dy1_f, de1_f, my1_f, res1_f = (
    e1[mask1], ee1[mask1], dy1[mask1], de1[mask1], my1[mask1], res1[mask1]
)
e2_f, ee2_f, dy2_f, de2_f, my2_f, res2_f = (
    e2[mask2], ee2[mask2], dy2[mask2], de2[mask2], my2[mask2], res2[mask2]
)

# masked arrays (preserve original length; zero bins masked)
e1_m  = np.ma.masked_where(~mask1, e1)
ee1_m = np.ma.masked_where(~mask1, ee1)
dy1_m = np.ma.masked_where(~mask1, dy1)
de1_m = np.ma.masked_where(~mask1, de1)
my1_m = np.ma.masked_where(~mask1, my1)
res1_m= np.ma.masked_where(~mask1, res1)

e2_m  = np.ma.masked_where(~mask2, e2)
ee2_m = np.ma.masked_where(~mask2, ee2)
dy2_m = np.ma.masked_where(~mask2, dy2)
de2_m = np.ma.masked_where(~mask2, de2)
my2_m = np.ma.masked_where(~mask2, my2)
res2_m= np.ma.masked_where(~mask2, res2)

In [29]:
%matplotlib notebook
plt.figure(figsize=(10, 8), dpi=100)

# Create subplots with custom height ratio (data plot taller than residuals)
gs = plt.GridSpec(2, 1, height_ratios=[3, 1])
ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

ax1.set_axisbelow(False)

# Spectrum 1 with filtered data (zeros removed)
ax1.errorbar(
    e1_f, dy1_f,
    fmt='o', ms=1,
    elinewidth=0.8, capsize=2,
    ls='-', lw=1, label='Data 1'
)

# Spectrum 2 with filtered data (zeros removed)
ax1.errorbar(
    e2_f, dy2_f,
    fmt='o', ms=1,
    elinewidth=0.6, capsize=1,
    color='green', alpha=0.6, ls='-', label='Data 2'
)

# Draw models using filtered arrays
ax1.step(e1_f, my1_f, where='mid', color='C1', lw=0.5, zorder=20, label='Model 1')
ax1.step(e2_f, my2_f, where='mid', color='darkgreen', lw=0.5, zorder=20, label='Model 2')

ax1.set_yscale("log")
ax1.set_ylabel(r"Counts s$^{-1}$ keV$^{-1}$")
ax1.set_title("Spectrum and Model")
ax1.legend()
ax1.set_xlim(1.0, 2.0)
ax1.set_ylim(1e-3, 10)

# Residuals subplot
ax2.plot(e1_f, res1_f, lw=0.5, label='Group 1')
ax2.plot(e2_f, res2_f, lw=0.5, label='Group 2', color='green')
ax2.axhline(0, color='red', linestyle='--')
ax2.legend()
ax2.set_xlim(1.0, 2.0)
ax2.set_xlabel("Energy (keV)")
ax2.set_ylabel(r"Residuals ($\chi$)")

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>